### Compute monthly and yearly averages of PFCLM outputs  
Notes (10/25):  
- last thing I was working on was figuring out how to loop though each month (with different numbers of days/month). I want to make it so that it reads through each month, saves the monthly averages, and moves onto the next month. Currently this is set up so that it takes in one month (`month_start = 'oct'`)
- fix the description below
- 




This script takes already postprocessed PFCLM daily averages as PFB files and computes monthly and yearly averages to be saved as PFB files.

**Inputs:**
- Directory where PFCLM daily averages are and directory where you want to save output (these should be the same)
- PFB files of PF and CLM daily averages
- water year and day start/end

**Outputs:**  
Monthly and yearly averages of PFCLM variables saved as PFB files:  
  - Overland flow at each grid cell (flow)
    - Soil moisture (SM)
    - Water table depth (WTD)

  - _Subsurface Storage_
    - Total Subsurface Storage (SUBstorage)
    - GW storage (GWstorage)
    - Soil moisture storage (SMstorage)

  - _Total Water Storage_
    - Surface water storage (SURF_WATstorage)
    - Total water storage (TWS)  
    
 - CLM variables  
   - Latent heat (LH) – CLM out layer 1 [W/m^2]
   - Sensible heat flux (SH) – CLM out layer 3 [W/m^2]
   - ground evaporation without condensation (qflx_grnd) – CLM out layer 6 [mm/s]
   - Vegetation transpiration (qflx_trans) – CLM out layer 9 [mm/s]
   - Snow water equivalent (SWE) – CLM out layer 11 [mm]
   - Ground temperature (Tgrnd) – CLM out layer 12 [K] skin temp
   - Soil temperature (Tsoil) – CLM out layer 14 [K] @5cm
   - Evapotranspiration calculation from qflx_evap_tot

In [1]:
import numpy as np
import pandas as pd
from parflow import Run
import sys
from parflow.tools.io import read_pfb,write_pfb
import parflow.tools.hydrology as hydro

In [54]:
water_year = 2003
day_start = 0 #day_start = 0 is the first day of the water year, Oct 1 (e.g., day_start = 2 starts at hour 49)
day_end = 3 #day_end = 364 is the final day of the water year, Sept 30

month_start = 'nov' #month you want to average variables for

# PFCLM run name
runname = f'Taylor_{water_year}' #f'CONUS2_{water_year}'

# Directory where averages are saved to
#directory_out = f'/glade/scratch/tijerina/CONUS2/spinup_WY2003/averages'
directory_out = '/glade/scratch/tijerina/CONUS2/spinup_WY2003/averages'

nz = 5 #10
ny = 47 #3256
nx = 45 #4442

dx = 1000
dy = 1000
dz = 5 # 200
#dz_3d = data.dz

# apparently it's good to use high numbers when saving files to speed up reading?
# for write_pfb function
p = 5 #72
q = 5 #48
r = 1

In [53]:
### Create dataframe of months and corresponding number of days for the Water Year, starting in October ###

months= ['oct', 'nov', 'dec', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep']
# days in water year starts with October (need to change 'days' if it's a leap year)
month_indexing = {
    "days": [31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30],
    "month_num": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
}

# load month_days into pandas dataframe
months_df = pd.DataFrame(month_indexing, index = months)

#Test some indexing, delete later
months_df['days'].loc[month_start]

31

In [41]:
#Test some indexing, delete later
for i in range(12):
    print(months_df['days'].loc[months[i]])

31
30
31
31
28
31
30
31
30
31
31
30


In [48]:
# initialize arrays
overland_flow_m = np.zeros((ny, nx)) # Flow
soil_moisture_m = np.zeros((nz,ny,nx)) # Soil Moisture
wtd_m = np.zeros((ny, nx)) # Water Table Depth    
surface_storage_m = np.zeros((ny,nx)) # Surface Water Storage
subsurface_storage_m = np.zeros((nz,ny,nx)) # Total Subsurface Storage


In [56]:
#for day in range(day_start,day_end):

#     timestamp_day_out = str(int(day+1)).rjust(3, '0')

#     ##INITIALIZE WHATEVER DYNAMIC VARIABLES THAT NEED HOURLY READING
#     overland_flow = np.zeros((ny, nx)) # Flow
#     soil_moisture = np.zeros((nz,ny,nx)) # Soil Moisture
#     wtd = np.zeros((ny, nx)) # Water Table Depth
    
#     surface_storage = np.zeros((ny,nx)) # Surface Water Storage
    
#     # Subsurface Storage Components
#     subsurface_storage = np.zeros((nz,ny,nx)) # Total Subsurface Storage
#     gw_storage = np.zeros((nz,ny,nx)) # Groundwater Storage
#     sm_storage = np.zeros((nz,ny,nx)) # Soil Moisture Storage
    
    
    
    
#     #if not variables_clm == False:
#     #    clm_output = np.zeros((NCLMOUTPUTS,ny,nx))


print(f'Computing averages for {month_start}')

for d in range((months_df['days'].loc[month_start])):
    timestamp_reading = str(int(d+1)).rjust(3, '0')
    print(timestamp_reading)
######## day*24+1,(day+1)*24+1



#     #read pressure and saturation at timestep 
#     saturation = read_pfb(f'{path_outputs}{runname}.out.satur.{timestamp_reading}.pfb') * active_mask
#     pressure = read_pfb(f'{path_outputs}{runname}.out.press.{timestamp_reading}.pfb') * active_mask
#     print(f'reading {path_outputs}{runname}.out... at time {timestamp_reading}')

#     ################### 
#     # Computations
#     ###################
#     # ##########Flow [m^3/s] DID NOT DO THIS HERE -> divide by 3600 to go from [m^3/h] to [m^3/s]
#     overland_flow = hydro.calculate_overland_flow_grid(pressure, slopex, slopey, mannings, dx, dy, mask = active_mask)
#     #Soil Moisture
#     soil_moisture += saturation * porosity
#     # Water Table Depth
#     wtd = hydro.calculate_water_table_depth(pressure, saturation, dz_3d)

#     # Surface Storage
#     ## total surface storage for this time step is the summation of substorage surface across all x/y slices <-- from other script, is this still TRUE??
#     surface_storage += hydro.calculate_surface_storage(pressure, dx, dy, mask = active_mask)

#     # Total Subsurface Storage
#     subsurface_storage += hydro.calculate_subsurface_storage(porosity, pressure, saturation, specific_storage, dx, dy, dz_3d, mask = active_mask)

#     # Groundwater Storage
#     #gw_storage +=

#     # Soil Moisture Storage
#     #sm_storage +=

Computing averages for nov
001
Computing averages for nov
002
Computing averages for nov
003
Computing averages for nov
004
Computing averages for nov
005
Computing averages for nov
006
Computing averages for nov
007
Computing averages for nov
008
Computing averages for nov
009
Computing averages for nov
010
Computing averages for nov
011
Computing averages for nov
012
Computing averages for nov
013
Computing averages for nov
014
Computing averages for nov
015
Computing averages for nov
016
Computing averages for nov
017
Computing averages for nov
018
Computing averages for nov
019
Computing averages for nov
020
Computing averages for nov
021
Computing averages for nov
022
Computing averages for nov
023
Computing averages for nov
024
Computing averages for nov
025
Computing averages for nov
026
Computing averages for nov
027
Computing averages for nov
028
Computing averages for nov
029
Computing averages for nov
030


In [ ]:
# yearly averages 
# This will work with reading in the daily PFB files (eg. see the compute_Total_Water_Storage.ipynb)

 ### Arrays for yearly averages ###
    overland_flow_y = np.zeros((ny, nx)) # Flow
    soil_moisture_y = np.zeros((nz,ny,nx)) # Soil Moisture
    wtd_y = np.zeros((ny, nx)) # Water Table Depth
    surface_storage_y = np.zeros((ny,nx)) # Surface Water Storage
    # Subsurface Storage Components
    subsurface_storage_y = np.zeros((nz,ny,nx)) # Total Subsurface Storage
    gw_storage_y = np.zeros((nz,ny,nx)) # Groundwater Storage
    sm_storage_y = np.zeros((nz,ny,nx)) # Soil Moisture Storage
 # yearly averages
        overland_flow_y += overland_flow / 365
        soil_moisture_y += soil_moisture / 365
        wtd_y += wtd / 365
        surface_storage_y += surface_storage / 365
        subsurface_storage_y += subsurface_storage / 365
        gw_storage_y += gw_storage / 365
        sm_storage_y += sm_storage / 365


In [ ]:
### compute average for average variables
overland_flow /= 24 #accumulated ####################################
soil_moisture /= 24
wtd /= 24 # CHANGE THIS TO BE ACCUMULATED?? 10/7/22

surface_storage /= 24

subsurface_storage /= 24
#gw_storage /= 24
#sm_storage /= 24


#subsurface[active_mask==0]=-10**(38) ### ???????????

### SAVE VARIABLES AS PFB FILES
write_pfb(f'{directory_out}/flow.{water_year}.monthly.{timestamp_day_out}.pfb',overland_flow,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
write_pfb(f'{directory_out}/SM.{water_year}.monthly.{timestamp_day_out}.pfb',soil_moisture,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
write_pfb(f'{directory_out}/WTD.{water_year}.monthly.{timestamp_day_out}.pfb',wtd,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
write_pfb(f'{directory_out}/SURF_WATstorage.{water_year}.monthly.{timestamp_day_out}.pfb',surface_storage,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
write_pfb(f'{directory_out}/SUBstorage.{water_year}.monthly.{timestamp_day_out}.pfb',subsurface_storage,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
#write_pfb(f'{directory_out}/GWstorage.{water_year}.monthly.{timestamp_day_out}.pfb',gw_storage,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
#write_pfb(f'{directory_out}/SMstorage.{water_year}.monthly.{timestamp_day_out}.pfb',sm_storage,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)



In [ ]:
for i in range(364):
    step = str(int(i+1)).rjust(3, '0')
    flow_pfb = np.squeeze(read_pfb(f'{directory_out}/flow.1999.daily.{step}.pfb'))
    daily_flow[i]=flow_pfb[15,23]
    print(step)